In [571]:
import lightning as pl
from torch.utils.data import DataLoader
import albumentations as A
import torchvision.transforms as T
import albumentations.pytorch as pytorch
import albumentations as albu

In [572]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torchmetrics import Accuracy, JaccardIndex, FBetaScore
from typing import Any, Union

In [573]:
# from pytorch_lightning.callbacks import ModelCheckpoint
# from pytorch_lightning.callbacks import EarlyStopping
# from pytorch_lightning.callbacks import LearningRateMonitor
# # from pytorch_lightning.loggers import TensorBoardLogger
from lightning.pytorch.callbacks import ModelCheckpoint,EarlyStopping,LearningRateMonitor, LearningRateFinder


import segmentation_models_pytorch as smp
import warnings

In [574]:
from torch.utils.data import Dataset

import pandas as pd
import numpy as np

import os

from PIL import Image

from segmentation_models_pytorch.utils import metrics

from segmentation_models_pytorch.losses import FocalLoss, DiceLoss, JaccardLoss

import re

import onnx
import onnxruntime

import pandas as pd
%matplotlib inline
# %matplotlib widget

import matplotlib as mpl
mpl.rc('font',family='Charter')

import matplotlib.pyplot as plt

import shutil


# from torchmetrics import BinaryConfusionMatrix
import random
random.seed(42)
torch.manual_seed(42)

In [575]:
import sys

sys.path.insert(1,"..")
from utils import read_binary_image

In [576]:
class ThermalDataset(Dataset):
    def __init__(self,
                 stage: str,
                 images_path: str,
                 augmentation: Any = None,
                 preprocessing: Any = None,
                 shuffle: bool = True,
                 seed: int = 42):

        self.__attribute_checking(images_path,
                                  stage, shuffle)

        self.images_path = images_path

        self.augmentation = augmentation
        self.preprocessing = preprocessing

        self.stage = stage
        self.shuffle = shuffle
        self.seed = seed
        self.total_len = None
        self._images, self._masks = self.__create_dataset()

        torch.manual_seed(seed)
        random.seed(seed)

    @staticmethod
    def __type_checking(images_path: str,
                        stage: str, shuffle: bool) -> None:
        assert isinstance(images_path, str)
        assert isinstance(stage, str)
        assert isinstance(shuffle, bool)



    @staticmethod
    def __path_checking(images_path: str) -> None:
        assert os.path.isdir(images_path)

    @staticmethod
    def __stage_checking(stage: str) -> None:
        assert stage in ["train", "test", "val"]

    @classmethod
    def __attribute_checking(cls, images_path: str,
                             stage: str,
                             shuffle: bool) -> None:

        cls.__type_checking(images_path=images_path,
                            stage=stage,
                            shuffle=shuffle)

        cls.__path_checking(images_path=images_path)

        cls.__stage_checking(stage=stage)

    def __create_dataset(self) -> dict:
        dict_paths = {
            "image": [],
            "mask": []
        }

        images_path = self.__split_data(self.stage)

        #### NEED TO ADD SHUFFLE ON HOW THE IMAGES ARE ACCESSED, AND INCLUDE SEED

        images_path_shuffle = os.listdir(images_path)
        
        random.shuffle(images_path_shuffle)

        for image_name in images_path_shuffle:
            dict_paths["image"].append(os.path.join(images_path,image_name))
            dict_paths["mask"].append(os.path.join(os.path.dirname(images_path),'masks',image_name.replace('_NIR_SWIR','_mask')))

        dataframe = pd.DataFrame(
            data=dict_paths,
            index=np.arange(0, len(dict_paths["image"]))
        )
        self.total_len = len(dataframe)
        data_dict = {self.stage: (dataframe["image"].values,dataframe["mask"].values)}

        return data_dict[self.stage]

    def __split_data(self, stage: str) -> str:
        return os.path.join(self.images_path,stage,'images')
    
    def __len__(self) -> int:
        return self.total_len

    def __getitem__(self, idx) -> tuple:

        # with open(self._images[idx],'rb') as image_file:
        #     image_bin = image_file.read()
        #     image = np.frombuffer(image_bin,dtype=np.float32).reshape(256,256,3)
        # with open(self._masks[idx],'rb') as mask_file:
        #     mask_bin = mask_file.read()
        #     mask = np.frombuffer(mask_bin,dtype=np.float32).reshape(256,256,1)

        image = read_binary_image(image_path=self._images[idx],
                                  dtype=np.float32,
                                  shape=[256,256,3])

        mask = read_binary_image(image_path=self._masks[idx],
                                  dtype=np.float32,
                                  shape=[256,256,1])

        # image = Image.open(self._images[idx])
        # mask = Image.open(self._masks[idx])
        
        # image = np.array(image)

        ### FOR FOCAL LOSS
        # mask = mask.convert('L') # This ensures that the label only have 1 band, which is necessary for binary classification
        # mask = np.array(mask)[:,:,np.newaxis]
        
        # mask = np.divide(mask,255).astype('float32') #Masks need to be 0-1 values
        
        # # apply augmentation
        if self.augmentation:
            sample = self.augmentation(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']
        
        # apply preprocessing
        if self.preprocessing:
            sample = self.preprocessing(image=image, mask=mask)
            image, mask = sample['image'], sample['mask']

        return image, mask


In [577]:
images_path = '/home/cristopher/Documents/SegTHRawS/datasets/train_geo_split_dataset'
ThermalDataset(stage = 'val',images_path=images_path)[0]

(array([[[0.01562882, 0.01636142, 0.01562882],
         [0.01587302, 0.01489621, 0.01538462],
         [0.01538462, 0.01636142, 0.01562882],
         ...,
         [0.01514042, 0.01562882, 0.01538462],
         [0.01538462, 0.01514042, 0.01538462],
         [0.01538462, 0.01538462, 0.01538462]],
 
        [[0.01611722, 0.01611722, 0.01562882],
         [0.01587302, 0.01587302, 0.01514042],
         [0.01514042, 0.01611722, 0.01587302],
         ...,
         [0.01538462, 0.01562882, 0.01514042],
         [0.01538462, 0.01562882, 0.01562882],
         [0.01587302, 0.01562882, 0.01587302]],
 
        [[0.01587302, 0.01562882, 0.01611722],
         [0.01636142, 0.01660562, 0.01660562],
         [0.01611722, 0.01562882, 0.01636142],
         ...,
         [0.01562882, 0.01587302, 0.01514042],
         [0.01562882, 0.01538462, 0.01587302],
         [0.01587302, 0.01514042, 0.01587302]],
 
        ...,
 
        [[0.01636142, 0.01562882, 0.01562882],
         [0.01587302, 0.01587302, 0.01562

In [578]:
def get_training_augmentation():
    train_transform = [

        albu.HorizontalFlip(p=0.5),
        albu.VerticalFlip(p=0.5),

    ]
    return albu.Compose(train_transform)

def to_tensor(x, **kwargs):
    return x.transpose(2, 0, 1).astype('float32')

def get_preprocessing(preprocessing_fn):
    """Construct preprocessing transform
    Args:
        preprocessing_fn (callbale): data normalization function 
            (can be specific for each pretrained neural network)
    Return:
        transform: albumentations.Compose
    
    """
    
    _transform = [
        albu.Lambda(image=preprocessing_fn),
        albu.Lambda(image=to_tensor, mask=to_tensor),
    ]
    return albu.Compose(_transform)

In [579]:
class ThermalDataModule(pl.LightningDataModule):
    def __init__(self,images_path: str,
                 augmentation: Union[T.Compose, A.Compose],
                 preprocessing: Any,
                 batch_size: int = 5,
                 num_workers: int = os.cpu_count(),
                 seed: int = 42):
        super().__init__()
        self.batch_size = batch_size
        self.num_workers = num_workers
        self.images_path = images_path
        self.data_train = None
        self.data_val = None
        self.data_test = None
        self.data_predict = None
        self.seed = seed

        self.train_augmentation = augmentation
        self.eval_augmentation = augmentation
        self.preprocessing = preprocessing


    def setup(self, stage: str = None) -> None:
        self.data_train = ThermalDataset(
            images_path=self.images_path,
            augmentation=self.train_augmentation,
            preprocessing=self.preprocessing,
            stage="train",
            shuffle=True,
            seed=self.seed
            )

        self.data_val = ThermalDataset(
            images_path=self.images_path,
            augmentation=self.eval_augmentation,
            preprocessing=self.preprocessing,
            stage="val",
            shuffle=True,
            seed=self.seed
            )

        self.data_test = ThermalDataset(
            images_path=self.images_path,
            augmentation=self.eval_augmentation,
            preprocessing=self.preprocessing,
            stage="test",
            shuffle=True,
            seed=self.seed
            )

        self.data_predict = ThermalDataset(
            images_path=self.images_path,
            augmentation=self.eval_augmentation,
            preprocessing=self.preprocessing,
            stage="test",
            shuffle=True,
            seed=self.seed
            )

    def train_dataloader(self) -> DataLoader:
        return DataLoader(
            dataset=self.data_train,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=True
        )

    def val_dataloader(self) -> DataLoader:
        return DataLoader(
            dataset=self.data_val,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False
        )

    def test_dataloader(self) -> DataLoader:
        return DataLoader(
            dataset=self.data_test,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False
        )

    def predict_dataloader(self) -> DataLoader:
        return DataLoader(
            dataset=self.data_predict,
            batch_size=self.batch_size,
            num_workers=self.num_workers,
            shuffle=False
        )

In [580]:
# class ThermalModel(pl.LightningModule):
#     def __init__(self,
#                  model: nn.Module,
#                  loss_fn: Any,
#                  optim_dict: dict = None,
#                  lr: float = None,
#                  num_classes: int = 1):
#         super().__init__()
#         self.save_hyperparameters(ignore=['model','loss_fn'])

#         self.num_classes = num_classes
#         self.model = model
#         # self.criterion = nn.CrossEntropyLoss()
#         self.criterion = loss_fn
#         self.optim_dict = optim_dict
#         self._device = "cuda" if torch.cuda.is_available else "cpu"

#         self.step_outputs = {
#             "loss": [],
#             "accuracy": [],
#             "jaccard_index": [],
#             "fbeta_score": [],
#             "IoU": []
#         }

#         self.metrics = {
#             "accuracy": Accuracy(task="binary",
#                                  threshold=0.5,
#                                  num_classes=num_classes,
#                                  validate_args=True,
#                                  ignore_index=None,
#                                  average="micro").to(self._device),

#             "jaccard_index": JaccardIndex(task="binary",
#                                           threshold=0.5,
#                                           num_classes=num_classes,
#                                           validate_args=True,
#                                           ignore_index=None,
#                                           average="macro").to(self._device),

#             "fbeta_score": FBetaScore(task="binary",
#                                       beta=1.0,
#                                       threshold=0.5,
#                                       num_classes=num_classes,
#                                       average="micro",
#                                       ignore_index=None,
#                                       validate_args=True).to(self._device),

#             "IoU": metrics.IoU()
#         }

#     def forward(self, x):
#         return self.model(x)

#     def shared_step(self, batch, stage: str) -> torch.Tensor:
#         x, y = batch
#         x, y = x.to(self._device),y.to(self._device)

#         assert x.ndim == 4
#         assert x.max() <= 3 and x.min() >= -3 
#         assert y.ndim == 4
#         assert y.max() <= 1 and y.min() >= 0

#         logits = self.forward(x.to(torch.float32))
        

#         # activated = F.softmax(input=logits, dim=1)
#         # predictions = torch.argmax(activated, dim=1)

#         predictions = torch.round(torch.sigmoid(logits))
#         # predictions = logits
        
#         loss = self.criterion(logits, y)
        
#         accuracy = self.metrics["accuracy"](predictions, y)
#         jaccard_index = self.metrics["jaccard_index"](predictions, y)
#         fbeta_score = self.metrics["fbeta_score"](predictions, y)
#         IoU_score = self.metrics["IoU"](predictions, y)

#         # print(f'stage: {stage}')
#         # print(f'Jaccard: {jaccard_index.dtype}')
#         # print(f'loss: {loss.dtype}')

#         self.step_outputs["loss"].append(loss)
#         self.step_outputs["accuracy"].append(accuracy)
#         self.step_outputs["jaccard_index"].append(jaccard_index)
#         self.step_outputs["fbeta_score"].append(fbeta_score)
#         self.step_outputs["IoU"].append(IoU_score)


#         self.log(f'{stage}_loss'   , loss          , prog_bar=True , on_step=False , on_epoch=True)
#         # self.log(f'{stage}_acc'    , accuracy      , prog_bar=True , on_step=False , on_epoch=True)
#         # self.log(f'{stage}_jaccard', jaccard_index , prog_bar=True , on_step=False , on_epoch=True)
#         self.log(f'{stage}_fbeta'  , fbeta_score   , prog_bar=True , on_step=False , on_epoch=True)
#         self.log(f'{stage}_IoU'    , IoU_score     , prog_bar=True , on_step=False , on_epoch=True)
        
#         return loss

#     def shared_epoch_end(self, stage: Any):
#         loss = torch.mean(torch.tensor([
#             loss for loss in self.step_outputs["loss"]
#         ]))

#         accuracy = torch.mean(torch.tensor([
#             accuracy for accuracy in self.step_outputs["accuracy"]
#         ]))

#         jaccard_index = torch.mean(torch.tensor([
#             jaccard_index for jaccard_index in self.step_outputs["jaccard_index"]
#         ]))

#         print(f'stage: {stage}')
#         print(f'jaccard: {self.step_outputs["jaccard_index"]}')
#         print(f'Result: {jaccard_index}')


#         fbeta_score = torch.mean(torch.tensor(
#             [fbeta_score for fbeta_score in self.step_outputs["fbeta_score"]
#              ]))

#         IoU_score = torch.mean(torch.tensor(
#                 [IoU_score for IoU_score in self.step_outputs["IoU"]
#                  ]))

#         for key in self.step_outputs.keys():
#             self.step_outputs[key].clear()

#         metrics = {
#             f"{stage}_loss": loss,
#             f"{stage}_accuracy": accuracy,
#             f"{stage}_jaccard_index": jaccard_index,
#             f"{stage}_fbeta_score": fbeta_score,
#             f"{stage}_IoU": IoU_score
#         }
#         self.log_dict(metrics, prog_bar=True)

#     def training_step(self, batch: Any, batch_idx: Any):
#         return self.shared_step(batch=batch, stage="train")

#     def on_train_epoch_end(self) -> None:
#         return self.shared_epoch_end(stage="train")

#     def validation_step(self, batch: Any, batch_idx: Any):
#         return self.shared_step(batch=batch, stage="val")

#     def on_validation_epoch_end(self) -> None:
#         return self.shared_epoch_end(stage="val")

#     def test_step(self, batch: Any, batch_idx: Any):
#         return self.shared_step(batch=batch, stage="test")

#     def on_test_epoch_end(self) -> None:
#         return self.shared_epoch_end(stage="test")

#     def predict_step(self, batch: Any, batch_idx: int, dataloader_idx: int = 0):
#         x, y = batch

#         assert x.ndim == 4
#         assert x.max() <= 3 and x.min() >= -3
#         assert y.ndim == 4
#         assert y.max() <= 1 and y.min() >= 0

#         logits = self.forward(x.to(torch.float32))
#         # predictions = logits
#         predictions = torch.round(torch.sigmoid(logits))

#         # activated = F.softmax(input=logits, dim=1)
#         # predictions = torch.argmax(activated, dim=1)

#         return predictions

#     def configure_optimizers(self):
#         optimizer = torch.optim.Adam(
#             params=self.parameters(),
#             lr=self.hparams.lr
#         )

#         scheduler_dict = {
#             "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(
#                 optimizer=optimizer,
#                 patience=5
#             ),
#             # "scheduler": pl_bolts.optim.lr_scheduler.LinearWarmupCOsineAnnealingLR(
#             #     optimizer=optimizer,
#             #     warmup_epochs=10,
#             #     max_epochs=30,
#             # ),
#             "interval": "epoch",
#             "monitor": "val_loss"
#         }
        
#         optimization_dictionary = {"optimizer": optimizer, "lr_scheduler": scheduler_dict}
#         return self.optim_dict if self.optim_dict else optimization_dictionary


In [581]:


class Combined_Focal_Dice_Loss(pl.LightningModule):
    '''
    Combined weighted loss between Focal Loss and Dice Loss  
    '''
    def __init__(self,
                 focal_loss_weight: float = 0.5,
                 dice_weight: float = None,
                 log_dice_loss: bool = False):
        
        super(Combined_Focal_Dice_Loss, self).__init__()
        
        self.focal_loss_weight = focal_loss_weight
        self.dice_weight = (1 - focal_loss_weight) if dice_weight is None else dice_weight

        if self.focal_loss_weight + self.dice_weight != 1:
            warnings.warn("Sum of Focal and Dice loss weights is not 1.0: "
                          f"{self.focal_loss_weight:.2f} + {self.dice_weight:.2f} = "
                          f"{self.focal_loss_weight + self.dice_weight:.2f}")

        self.log_dice_loss = log_dice_loss


    # def dice_score(y_pred, y_true, eps=1e-15, smooth=1.):
    #     intersection = (y_pred * y_true).sum()
    #     union = y_pred.sum() + y_true.sum()
    #     return (2. * intersection + smooth) / (union + smooth + eps)


    def forward(self, y_pred, y_true):

        focal_loss_fn = FocalLoss(mode= 'binary')
        focal_loss_fn.__name__ = 'focal_loss'
        dice_loss_fn = DiceLoss(mode= 'binary',from_logits=True,log_loss=self.log_dice_loss) #Typically Dice use the masks and not logits, that is why from logits is used because y_pred are the logits
        dice_loss_fn.__name__ = 'dice_loss'


        focal_loss = focal_loss_fn(y_pred, y_true) 
        dice_loss = dice_loss_fn(y_pred, y_true) 
        
        # y_pred = torch.sigmoid(y_pred)
        # dice_loss = 1- dice_score(y_pred, y_true)
        # log_dice_loss = -torch.log(dice_score(y_pred, y_true))
        
        loss = self.focal_loss_weight * focal_loss + self.dice_weight * dice_loss

        return loss
    


In [599]:

class ThermalModel(pl.LightningModule):
    def __init__(self,
                 model: nn.Module,
                 loss_fn: Any,
                 lr: float = 3e-4,
                 num_classes: int = 1,
                 batch_size: int = 16):
        super().__init__()
        self.save_hyperparameters(ignore=['model','loss_fn'])

        

        self.num_classes = num_classes
        self.model = model
        # self.criterion = nn.CrossEntropyLoss()
        self.criterion = loss_fn
        self._device = "cuda" if torch.cuda.is_available else "cpu"
        # self.learning_rate = lr

        self.train_outputs = {
            "loss": [],
            "fbeta_score": [],
            "IoU": []
        }

        self.val_outputs = {
            "loss": [],
            "fbeta_score": [],
            "IoU": []
        }

        self.example_input_array = torch.zeros((1,3,256,256),dtype = torch.float32)


        # self.step_outputs = {
        #     "tp": [],
        #     "tn": [],
        #     "fp": [],
        #     "fn": []
        # }

        # self.train_tp = []
        # self.train_tn = []
        # self.train_fp = []
        # self.train_fn = []
        
        # self.val_tp = []       
        # self.val_tn = []
        # self.val_fp = []
        # self.val_fn = []
        
        
        # self.stage_outputs = {
        #     "train": self.step_outputs,
        #     "val": self.step_outputs,
        #     "test": self.step_outputs
        # }

        self.metrics = {
            "accuracy": Accuracy(task="binary",
                                 threshold=0.5,
                                 num_classes=num_classes,
                                 validate_args=True,
                                 ignore_index=None,
                                 average="micro").to(self._device),

            "jaccard_index": JaccardIndex(task="binary",
                                          threshold=0.5,
                                          num_classes=num_classes,
                                          validate_args=True,
                                          ignore_index=None,
                                          average="macro").to(self._device),

            "fbeta_score": FBetaScore(task="binary",
                                      beta=1.0,
                                      threshold=0.5,
                                      num_classes=num_classes,
                                      average="micro",
                                      ignore_index=None,
                                      validate_args=True).to(self._device),

            "IoU": metrics.IoU()
        }

    def forward(self, x):

        return self.model(x)

    # def shared_step(self, batch, stage: str) -> torch.Tensor:
    #     x, y = batch
    #     x, y = x.to(self._device),y.to(self._device)

    #     assert x.ndim == 4
    #     assert x.max() <= 3 and x.min() >= -3 
    #     assert y.ndim == 4
    #     assert y.max() <= 1 and y.min() >= 0

    #     logits = self.forward(x.to(torch.float32))
        
    #     # tensorboard = self.logger.experiment
    #     # # tensorboard.add_graph(self.model.state_dict)
    #     # tensorboard.add_image('a',y[0],0)



    #     # testing  = (torch.softmax(logits, dim=0))
    #     testing  = (torch.sigmoid(logits))
    #     predictions = (testing > 0.5).float()


    #     # tp, fp, fn, tn  = smp.metrics.get_stats(predictions.long(),y.long(),mode = 'binary')

    #     # print(predictions)
    #     # print(tp,fp,fn,tn)

    #     # confmat = BinaryConfusionMatrix()(predictions.detach().cpu(),y.detach().cpu().float())
    #     # print(confmat)

    #     # print(torch.sum(tp).detach().cpu().numpy(),torch.sum(tn).detach().cpu().numpy(),torch.sum(fp).detach().cpu().numpy(),torch.sum(fn).detach().cpu().numpy())
        
    #     loss = self.criterion(logits, y)
        

    #     # print(stage)

    #     # accuracy = self.metrics["accuracy"](predictions, y)
    #     # jaccard_index = self.metrics["jaccard_index"](predictions, y)
    #     fbeta_score = self.metrics["fbeta_score"](predictions, y)
    #     IoU_score = self.metrics["IoU"](predictions, y)

    #     # self.log(f'{stage}_loss'   , loss          , prog_bar=True , on_step=False , on_epoch=True)
    #     # self.log(f'{stage}_fbeta'  , fbeta_score   , prog_bar=True , on_step=False , on_epoch=True)
    #     # self.log(f'{stage}_IoU'    , IoU_score     , prog_bar=True , on_step=False , on_epoch=True)


    #     metrics = {
    #         f"{stage}_loss_step": loss,
    #         f"{stage}_fbeta_score_step": fbeta_score,
    #         f"{stage}_IoU_step": IoU_score,
    #     }
    #     self.log_dict(metrics, prog_bar=True)

    #     self.step_outputs["loss"].append(loss)
    #     self.step_outputs["fbeta_score"].append(fbeta_score)
    #     self.step_outputs["IoU"].append(IoU_score)

        
    #     return loss
        # print(f'stage: {stage}')
        # print(f'Jaccard: {jaccard_index.dtype}')
        # print(f'loss: {loss.dtype}')
        
        # if torch.any(y!=0): # This avoids problems with the empty masks, that provides or 1 IoU or nan Jaccard Index 

        #     # self.stage_outputs[stage]["loss"].append(loss)
        #     # self.stage_outputs[stage]["accuracy"].append(accuracy)
        #     # self.stage_outputs[stage]["jaccard_index"].append(jaccard_index)
        #     # self.stage_outputs[stage]["fbeta_score"].append(fbeta_score)
        #     # self.stage_outputs[stage]["IoU"].append(IoU_score)


        #     self.log(f'{stage}_loss'   , loss          , prog_bar=True , on_step=False , on_epoch=True)
        #     self.log(f'{stage}_acc'    , accuracy      , prog_bar=True , on_step=False , on_epoch=True)
        #     self.log(f'{stage}_jaccard', jaccard_index , prog_bar=True , on_step=False , on_epoch=True)
        #     self.log(f'{stage}_fbeta'  , fbeta_score   , prog_bar=True , on_step=False , on_epoch=True)
        #     self.log(f'{stage}_IoU'    , IoU_score     , prog_bar=True , on_step=False , on_epoch=True)

            # self.log(f'{stage}_tp'     , tp            , prog_bar=True , on_step=False , on_epoch=True)
            # self.log(f'{stage}_fp'     , fp            , prog_bar=True , on_step=False , on_epoch=True)
            # self.log(f'{stage}_fn'     , fn            , prog_bar=True , on_step=False , on_epoch=True)
            # self.log(f'{stage}_tn'     , tn            , prog_bar=True , on_step=False , on_epoch=True)

        # else:
            # print(stage,loss,jaccard_index,IoU_score,fbeta_score)
            # for i in range(5):
            #     plt.figure()
            #     plt.subplot(1,2,1)
            #     plt.imshow(predictions.detach().cpu().numpy().squeeze()[i,:,:])
            #     plt.subplot(1,2,2)
            #     plt.imshow(y.detach().cpu().numpy().squeeze()[i,:,:])
            #     plt.show()


        # self.log(f'{stage}_loss'   , loss          , prog_bar=True , on_step=False , on_epoch=True)
        # self.log(f'{stage}_fbeta'  , fbeta_score   , prog_bar=True , on_step=False , on_epoch=True)
        # self.log(f'{stage}_IoU'    , IoU_score     , prog_bar=True , on_step=False , on_epoch=True)



        # if stage == 'train':
        #     self.train_tp.append(torch.sum(tp).detach().cpu().numpy())
        #     self.train_tn.append(torch.sum(tn).detach().cpu().numpy())
        #     self.train_fp.append(torch.sum(fp).detach().cpu().numpy())
        #     self.train_fn.append(torch.sum(fn).detach().cpu().numpy())
            
        # if stage == 'val':
        #     self.val_tp.append(torch.sum(tp).detach().cpu().numpy())
        #     self.val_tn.append(torch.sum(tn).detach().cpu().numpy())
        #     self.val_fp.append(torch.sum(fp).detach().cpu().numpy())
        #     self.val_fn.append(torch.sum(fn).detach().cpu().numpy())

        # self.stage_outputs[stage]['tp'].append(torch.sum(tp).detach().cpu().numpy())
        # self.stage_outputs[stage]['tn'].append(torch.sum(tn).detach().cpu().numpy())
        # self.stage_outputs[stage]['fp'].append(torch.sum(fp).detach().cpu().numpy())
        # self.stage_outputs[stage]['fn'].append(torch.sum(fn).detach().cpu().numpy())

        # self.log(f'{stage}_tp'     , tp            , prog_bar=True , on_step=False , on_epoch=True)
        # self.log(f'{stage}_fp'     , fp            , prog_bar=True , on_step=False , on_epoch=True)
        # self.log(f'{stage}_fn'     , fn            , prog_bar=True , on_step=False , on_epoch=True)
        # self.log(f'{stage}_tn'     , tn            , prog_bar=True , on_step=False , on_epoch=True)

        
        # # self.optimizers().step()
        # self.lr_schedulers().step()


        # return loss

        # return {
        #     "loss": loss,
        #     "tp": tp,
        #     "fp": fp,
        #     "fn": fn,
        #     "tn": tn,
        # }

    # def shared_epoch_end(self, stage: Any):

    #     print(stage)

    #     loss = torch.mean(torch.tensor([
    #         loss for loss in self.step_outputs["loss"]
    #     ]))

    #     print(self.step_outputs)

    #     fbeta_score = torch.mean(torch.tensor(
    #         [fbeta_score for fbeta_score in self.step_outputs["fbeta_score"]
    #          ]))

    #     metrics = {
    #         f"{stage}_loss": loss,
    #         f"{stage}_fbeta_score": fbeta_score
    #     }
    #     self.log_dict(metrics, prog_bar=True)

    #     for key in self.step_outputs.keys():
    #         self.step_outputs[key].clear()




    def training_step(self, batch: Any, batch_idx: Any):

        stage = 'train'

        x, y = batch
        x, y = x.to(self._device),y.to(self._device)

        assert x.ndim == 4
        assert x.max() <= 3 and x.min() >= -3 
        assert y.ndim == 4
        assert y.max() <= 1 and y.min() >= 0

        logits = self.forward(x.to(torch.float32))
        
        # tensorboard = self.logger.experiment
        # # tensorboard.add_graph(self.model.state_dict)
        # tensorboard.add_image('a',y[0],0)



        # testing  = (torch.softmax(logits, dim=0))
        testing  = (torch.sigmoid(logits))
        predictions = (testing > 0.5).float()
        loss = self.criterion(logits, y)
        
        fbeta_score = self.metrics["fbeta_score"](predictions, y)
        IoU_score = self.metrics["IoU"](predictions, y)

        metrics = {
            f"{stage}_loss_step": loss,
            f"{stage}_fbeta_score_step": fbeta_score,
            f"{stage}_IoU_step": IoU_score
        }
        self.log_dict(metrics, prog_bar=True)

        self.train_outputs["loss"].append(loss)
        self.train_outputs["fbeta_score"].append(fbeta_score)
        self.train_outputs["IoU"].append(IoU_score)

        
        return loss

        # return self.shared_step(batch=batch, stage="train")

    def on_train_epoch_end(self) -> None:

        print('train')


        loss = torch.mean(torch.tensor([
            loss for loss in self.train_outputs["loss"]
        ]))


        fbeta_score = torch.mean(torch.tensor(
            [fbeta_score for fbeta_score in self.train_outputs["fbeta_score"]
             ]))

        metrics = {
            f"{stage}_loss": loss,
            f"{stage}_fbeta_score": fbeta_score
        }
        self.log_dict(metrics, prog_bar=True)

        for key in self.train_outputs.keys():
            self.train_outputs[key].clear()
        # return loss

        # return self.shared_epoch_end(stage="train")

    def validation_step(self, batch: Any, batch_idx: Any):


        stage = 'val'


        x, y = batch
        x, y = x.to(self._device),y.to(self._device)

        assert x.ndim == 4
        assert x.max() <= 3 and x.min() >= -3 
        assert y.ndim == 4
        assert y.max() <= 1 and y.min() >= 0

        logits = self.forward(x.to(torch.float32))
        
        # tensorboard = self.logger.experiment
        # # tensorboard.add_graph(self.model.state_dict)
        # tensorboard.add_image('a',y[0],0)



        # testing  = (torch.softmax(logits, dim=0))
        testing  = (torch.sigmoid(logits))
        predictions = (testing > 0.5).float()
        loss = self.criterion(logits, y)
        
        fbeta_score = self.metrics["fbeta_score"](predictions, y)
        IoU_score = self.metrics["IoU"](predictions, y)

        metrics = {
            f"{stage}_loss_step": loss,
            f"{stage}_fbeta_score_step": fbeta_score,
            f"{stage}_IoU_step": IoU_score
        }
        self.log_dict(metrics, prog_bar=True)

        self.val_outputs["loss"].append(loss)
        self.val_outputs["fbeta_score"].append(fbeta_score)
        self.val_outputs["IoU"].append(IoU_score)

        
        return loss

        # return self.shared_step(batch=batch, stage="val")

    def on_validation_epoch_end(self) -> None:

        print('val')


        loss = torch.mean(torch.tensor([
            loss for loss in self.val_outputs["loss"]
        ]))


        fbeta_score = torch.mean(torch.tensor(
            [fbeta_score for fbeta_score in self.val_outputs["fbeta_score"]
             ]))

        metrics = {
            f"{stage}_loss": loss,
            f"{stage}_fbeta_score": fbeta_score
        }
        self.log_dict(metrics, prog_bar=True)

        for key in self.val_outputs.keys():
            self.val_outputs[key].clear()
        # return loss
        # return self.shared_epoch_end(stage="val")

    # def test_step(self, batch: Any, batch_idx: Any):
    #     return self.shared_step(batch=batch, stage="test")

    # def on_test_epoch_end(self) -> None:
    #     return self.shared_epoch_end(stage="test")

    def predict_step(self, batch: Any, batch_idx: int, dataloader_idx: int = 0):
        x, y = batch

        assert x.ndim == 4
        assert x.max() <= 3 and x.min() >= -3
        assert y.ndim == 4
        assert y.max() <= 1 and y.min() >= 0

        logits = self.forward(x.to(torch.float32))
        # predictions = torch.round(logits)
        # predictions = torch.round(torch.sigmoid(logits))
        
        prob_mask = logits.sigmoid()
        predictions = (prob_mask > 0.5).float()
        # predictions = (logits > 0.5).float()

        # activated = F.softmax(input=logits, dim=1)
        # predictions = torch.argmax(activated, dim=1)

        return predictions

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(
            params=self.parameters(),
            lr=self.hparams.lr
        )

        scheduler_dict = {
            "scheduler": torch.optim.lr_scheduler.ReduceLROnPlateau(
                optimizer=optimizer,
                patience=5
            ),
            # 'scheduler': torch.optim.lr_scheduler.CosineAnnealingLR(
            #     optimizer=optimizer,
            #     T_max=2,
            #     eta_min=0.0009
            # ),
            # "scheduler": pl_bolts.optimizers.lr_scheduler.LinearWarmupCosineAnnealingLR(
            #     optimizer=optimizer,
            #     warmup_epochs=2,
            #     max_epochs=3,
            #     eta_min = 0.001
            # ),
            # "interval": "step",
            
            "interval": "epoch",
            "monitor": "val_loss"
        }
        
        optimization_dictionary = {"optimizer": optimizer, "lr_scheduler": scheduler_dict}
        return optimization_dictionary


In [583]:
# from torch import nn


# class Activation(nn.Module):
#     def __init__(self, activation, **params):
#         super().__init__()

#         if activation is None or activation == "identity":
#             self.activation = nn.Identity(**params)
#         elif activation == "sigmoid":
#             self.activation = nn.Sigmoid()
#         elif activation == "softmax2d":
#             self.activation = nn.Softmax(dim=1, **params)
#         elif activation == "softmax":
#             self.activation = nn.Softmax(**params)
#         elif activation == "logsoftmax":
#             self.activation = nn.LogSoftmax(**params)
#         elif activation == "tanh":
#             self.activation = nn.Tanh()
#         elif callable(activation):
#             self.activation = activation(**params)
#         else:
#             raise ValueError(
#                 f"Activation should be callable/sigmoid/softmax/logsoftmax/tanh"
#                 f"/None; got {activation}"
#             )

#     def forward(self, x):
#         return self.activation(x)
    





In [584]:
def main(callbacks: list,
         model: Union[list, tuple],
         loss_fn: Any,
         augmentation: Any,
         preprocessing: Any,
         logger: Any,
         images_path: str,
         optim_dict: dict,
         min_epochs: int,
         max_epochs: int,
         batch_size: int = 16,
         precision: str = '16-mixed'
         ) -> None:

    # Trainer
    trainer = pl.Trainer(
        fast_dev_run=False,
        accelerator="auto",
        strategy="auto",
        devices="auto",
        num_nodes=1,
        logger=logger,
        callbacks=callbacks,
        max_epochs=max_epochs,
        min_epochs=min_epochs,
        num_sanity_val_steps=0,
        precision=precision # Mixed precision training
    )

    # Datamodule
    datamodule = ThermalDataModule(
        images_path=images_path,
        augmentation=augmentation,
        preprocessing=preprocessing,
        batch_size=batch_size,
        num_workers=os.cpu_count()
    )

    # LightningModule
    lightning_model = ThermalModel(
        model=model,
        loss_fn=loss_fn,
        # lr=3e-4
        batch_size=batch_size,
    )

    # Start training
    trainer.fit(model=lightning_model, datamodule=datamodule)

In [585]:
os.listdir(os.path.dirname(os.path.dirname(os.getcwd())))

['SegTHRawS',
 '.git',
 'README.md',
 'datasets',
 'requirements.txt',
 'models',
 'tree_segthraws_dataset.txt',
 'tree_datasets.txt',
 'new_THRawS_images',
 'segthraws_module']

In [586]:
# Run Constants
SEED: int = 42
ACTION: str = "ignore"
# DATA_PATH: str = os.path.join(os.getcwd(),'train_dataset')
# DATA_PATH: str = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'train_dataset')
DATA_PATH: str = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'datasets','train_geo_split_dataset')
# DATA_PATH: str = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())),'train_geo_split_dataset')
# DATA_PATH: str = os.path.join(os.getcwd(),'train_geo_split_dataset')
CHECKPOINT: Any = None
    
# Model Constants
CLASSES = 1
IN_CHANNELS = 3

optim_dict = None

# ENCODER = 'se_resnext50_32x4d'
ENCODER = 'mobilenet_v2'
# ENCODER = 'resnet18'
# ENCODER = 'timm-mobilenetv3_large_100'
ENCODER_WEIGHTS = 'imagenet'
    


ACTIVATION = None
# ACTIVATION = 'sigmoid' # could be None for logits. If used, the sigmoid after the forward function needs to be removed
DEVICE = 'cuda'

min_epochs = 150
max_epochs = 200

n_cpu = os.cpu_count()

model_name = 'Unet'
# model_name = 'DeepLabV3Plus'

gamma = 3

model_name_path = os.path.join(os.getcwd(),'models',f'{model_name}_{ENCODER}_focal_loss{gamma}_testing')
os.makedirs(model_name_path,exist_ok=True)  
run_idx =sum(1 for file in os.listdir(model_name_path) if file.startswith('run'))

model_main_path = os.path.join(model_name_path,f'run_{run_idx}')
os.makedirs(model_main_path,exist_ok=True)

# print(model_main_path)

# model_main_path = os.path.join(os.getcwd(),'models',f'{model_name}_{ENCODER}_{run_idx}')
metrics_path = os.path.join(model_main_path,'metrics')
os.makedirs(metrics_path,exist_ok=True)

# model = smp.DeepLabV3Plus(
#     encoder_name=ENCODER, 
#     encoder_weights=ENCODER_WEIGHTS, 
#     classes=1, 
#     activation=ACTIVATION,
# )

model = smp.Unet(
    encoder_name=ENCODER, 
    encoder_weights=ENCODER_WEIGHTS, 
    in_channels = 3,
    classes=CLASSES, 
    activation=ACTIVATION,
)

preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

precision = '16-mixed' # 32 # 32 is the original, and 16 is mixed precission
# precision = 32

loss = FocalLoss(mode= 'binary',gamma=gamma)
loss.__name__ = 'focal_loss'

# # loss = DiceLoss(mode= 'binary')
# # loss.__name__ = 'dice_loss'

# loss = JaccardLoss(mode= 'binary')
# loss.__name__ = 'jaccard_loss'

# # loss = losses.DiceLoss()
# # loss = losses.JaccardLoss()

# metrics = [
#     metrics.IoU(),
# ]

# optimizer = torch.optim.Adam([ 
#     dict(params=model.parameters(), lr=1e-3),
# ])

augmentation=get_training_augmentation()
preprocessing=get_preprocessing(preprocessing_fn)


batch_size = 16


In [587]:
# import matplotlib.image as mpimg
# import segmentation_models_pytorch as smp


# ENCODER = 'mobilenet_v2'
# ENCODER_WEIGHTS = 'imagenet'

# preprocessing_fn = smp.encoders.get_preprocessing_fn(ENCODER, ENCODER_WEIGHTS)

# preprocessing=get_preprocessing(preprocessing_fn)


# image_path = '/home/cristopher/Documents/SegTHRawS_training/model_training/train_dataset/test/images/Australia_1_G1_(384, 0, 640, 256)_NIR_SWIR.png'

# mean_mobilenet_v2 =  [0.485, 0.456, 0.406]
# std_mobilenet_v2 =  [0.229, 0.224, 0.225]

# # Define the directory containing the PNG images
# image_directory = os.path.join(os.getcwd(),'inputImages')

# image = mpimg.imread(image_path)

# sample = preprocessing(image=image)
# image_processed = sample['image']

# # print(np.transpose((image-mean_mobilenet_v2)/std_mobilenet_v2,(2,0,1)))

# # print(image_processed)


In [588]:
# Callbacks
callbacks = [
    ModelCheckpoint(
        dirpath=model_main_path,
        filename=f"{model_name}_{ENCODER}_"+"{epoch}",
        save_top_k=10,
        monitor="val_loss",
        mode="min"
    ),

    EarlyStopping(
        monitor="val_loss",
        min_delta=1e-5,
        patience=8,
        verbose=False,
        mode="min"
    ),

    LearningRateMonitor(
        logging_interval="step"
    ),

    # LearningRateFinder(
    #     min_lr = 1e-5,
    #     max_lr = 1e-2,
    # )
    
]


In [589]:
from lightning.pytorch.loggers import TensorBoardLogger
logger = TensorBoardLogger(save_dir="./logs", name=model_name,log_graph=True,default_hp_metric=False)



# from lightning.pytorch.loggers import CSVLogger
# logger = CSVLogger(f"{model_main_path}/csv_logs", name=f"{model_name}_{ENCODER}")


In [590]:

# # Trainer
# trainer = pl.Trainer(
#     fast_dev_run=False,
#     accelerator="auto",
#     strategy="auto",
#     devices="auto",
#     num_nodes=1,
#     logger=logger,
#     callbacks=callbacks,
#     max_epochs=max_epochs,
#     min_epochs=min_epochs,
#     precision=precision # Mixed precision training
# )
# # # Datamodule
# datamodule = ThermalDataModule(
#     images_path=images_path,
#     augmentation=augmentation,
#     preprocessing=preprocessing,
#     batch_size=batch_size,
#     num_workers=os.cpu_count()
# )

# # LightningModule
# lightning_model = ThermalModel(
#     model=model,
#     loss_fn=loss,
#     optim_dict=optim_dict,
# )

# from lightning.pytorch.tuner import Tuner

# tuner = Tuner(trainer)

# lr_finder = tuner.lr_find(lightning_model,datamodule=datamodule)

# new_lr = lr_finder.suggestion()
# lightning_model.hparams.lr = new_lr
# # # Start training
# trainer.fit(model=lightning_model, datamodule=datamodule)

In [591]:
# fig = lr_finder.plot(suggest=True)
# fig.show()
# # fig.savefig()

In [592]:
# final_lr = tuner.lr_find(lightning_model,datamodule=datamodule)

In [593]:
# fig = final_lr.plot(suggest=True)
# fig.show()

# new_lr = final_lr.suggestion()

# # model.hparams.lr = new_lr
# # model.hparams.lr = new_lr
# print(new_lr)

In [594]:
# batch_size_finder = tuner.scale_batch_size(model=lightning_model,datamodule=datamodule,mode='binsearch')

In [595]:
# batch_size_finder

In [596]:
main(
    callbacks=callbacks,
    model=model,
    loss_fn= loss, #Combined_Focal_Dice_Loss(),
    augmentation=augmentation,
    preprocessing=preprocessing,
    logger=logger,
    images_path=DATA_PATH,
    optim_dict=optim_dict,
    min_epochs=2, #min_epochs,
    max_epochs=2, #max_epochs
    batch_size=60,
    precision='16-mixed' 
)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/cristopher/.local/lib/python3.10/site-packages/lightning/pytorch/callbacks/model_checkpoint.py:653: Checkpoint directory /home/cristopher/Documents/SegTHRawS/SegTHRawS/model_training/models/Unet_mobilenet_v2_focal_loss3_testing/run_4 exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type      | Params | In sizes         | Out sizes       
------------------------------------------------------------------------------
0 | model     | Unet      | 6.6 M  | [1, 3, 256, 256] | [1, 1, 256, 256]
1 | criterion | FocalLoss | 0      | ?                | ?               
------------------------------------------------------------------------------
6.6 M     Trainable params
0         Non-trainable params
6.6 M     Total params
26.516    Total estimated model params

Epoch 0: 100%|██████████| 24/24 [00:06<00:00,  3.51it/s, v_num=4, train_loss_step=0.0176, train_fbeta_score_step=0.116, train_IoU_step=0.0613]   val
{'loss': [tensor(0.1763, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.1295, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.1115, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0961, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0837, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0736, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0656, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0587, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0529, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0487, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0440, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0404, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0364, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0328, device='cuda:0', grad_fn=<MeanBackward0>), tensor(0.0303, device='cuda:0

`Trainer.fit` stopped: `max_epochs=2` reached.


Epoch 1: 100%|██████████| 24/24 [00:09<00:00,  2.46it/s, v_num=4, train_loss_step=0.00833, train_fbeta_score_step=0.00812, train_IoU_step=0.00407, val_loss_step=0.00767, val_fbeta_score_step=0.0829, val_IoU_step=0.0433, val_loss=0.0101, val_fbeta_score=0.181, train_loss=nan.0, train_fbeta_score=nan.0]


In [ ]:
model.state_dict()

In [333]:
# %load_ext tensorboard
# # %tensorboard --logdir models/current_best_model/version_33

# %tensorboard --logdir logs/Unet/version_2

In [334]:

# checkpoints_paths = [os.path.join(model_main_path,checkpoint_path) for checkpoint_path in os.listdir(model_main_path) if checkpoint_path[-5:]=='.ckpt']
# checkpoint_path = max(checkpoints_paths, key=lambda x: int(re.search(r'epoch=(\d+)', x).group(1)))

# for checkpoint in checkpoints_paths:
#     if checkpoint != checkpoint_path:
#         os.remove(checkpoint)



In [ ]:
# # ### Perform the testing, c  NEED TO create a function
# trained_model = ThermalModel.load_from_checkpoint(checkpoint_path=checkpoint_path,model=model,loss_fn=loss)
# trained_model.eval();

# trainer = pl.Trainer(
#     fast_dev_run=False,
#     accelerator="auto",
#     strategy="auto",
#     devices="auto",
#     num_nodes=1,
#     logger=logger,
#     callbacks=callbacks,
#     max_epochs=1,
#     min_epochs=1,
#     precision=precision #Mixed precision training
# )

# # Datamodule
# datamodule = ThermalDataModule(
#     images_path=DATA_PATH,
#     augmentation=augmentation,
#     preprocessing=preprocessing,
#     batch_size=12,
#     num_workers=os.cpu_count()
# )

# loss_2 = FocalLoss(mode= 'binary')
# loss_2.__name__ = 'focal_loss'

# # LightningModule
# lightning_model = ThermalModel(
#     model=model,
#     loss_fn=loss_2,
#     optim_dict=optim_dict,
#     lr=3e-4
# )

# test_metrics = trainer.test(model=trained_model,datamodule=datamodule)[0]
# # trainer.predict(model=trained_model,datamodule=datamodule)

In [ ]:



# class ThermalModel_sigmoid(pl.LightningModule):
class ThermalModel_sigmoid(pl.LightningModule):
    def __init__(self,
                 model: pl.LightningModule,
                 activation: Any = 'sigmoid'):
        super().__init__()
        
        

        self.model = model
        self.metrics  = model.metrics
        self._device = "cuda" if torch.cuda.is_available else "cpu"

        if activation == "sigmoid":
            self.activation = nn.Sigmoid()

    # def Activation(self, activation, **params):
    #     if activation == "sigmoid":
    #         self.activation = nn.Sigmoid()

        # elif activation == "softmax2d":
        #     self.activation = nn.Softmax(dim=1, **params)
        # elif activation == "softmax":
        #     self.activation = nn.Softmax(**params)
        # elif activation == "logsoftmax":
        #     self.activation = nn.LogSoftmax(**params)
        # elif activation == "tanh":
        #     self.activation = nn.Tanh()
        else:
            raise ValueError(
                f"Activation should be callable/sigmoid/softmax/logsoftmax/tanh"
                f"/None; got {activation}"
            )

    def forward(self, x):

        self.model.eval()
        with torch.no_grad():
            x = self.model(x)

            output = (self.activation(x)>0.5).float()

        return output



    def test_step(self, batch: Any, batch_idx: Any):

        x, y = batch
        x, y = x.to(self._device),y.to(self._device)

        assert x.ndim == 4
        assert x.max() <= 3 and x.min() >= -3 
        assert y.ndim == 4
        assert y.max() <= 1 and y.min() >= 0

        predictions = self.forward(x.to(torch.float32))
        
        stage = 'test'

        fbeta_score = self.metrics["fbeta_score"](predictions, y)
        IoU_score = self.metrics["IoU"](predictions, y)


        self.log(f'{stage}_fbeta'  , fbeta_score   , prog_bar=True , on_step=False , on_epoch=True)
        self.log(f'{stage}_IoU'    , IoU_score     , prog_bar=True , on_step=False , on_epoch=True)

        return predictions

new_model = ThermalModel_sigmoid(model=trained_model,activation='sigmoid')
new_model

# import matplotlib.image as mpimg

# image  = mpimg.imread('train_dataset/test/images/Australia_0_G0_(0, 384, 256, 640)_NIR_SWIR.png')

# image_model  = np.transpose(image,(2,0,1))[np.newaxis]


# new_model(image_model)


# torch.save(new_model,os.path.join(model_main_path,'new_model'))


ThermalModel_sigmoid(
  (model): ThermalModel(
    (model): Unet(
      (encoder): MobileNetV2Encoder(
        (features): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): InvertedResidual(
            (conv): Sequential(
              (0): Conv2dNormActivation(
                (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU6(inplace=True)
              )
              (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
          )
        

In [ ]:
# trained_model = ThermalModel.load_from_checkpoint(checkpoint_path='models/Unet_mobilenet_v2_metrics_without_empty_masks/Unet_mobilenet_v2_epoch=149.ckpt',model=model,loss_fn=loss)
# trained_model.eval();


# trained_model = new_model

# x = torch.randn(1, 3, 256, 256).cpu()
# model_onnx = trained_model.cpu()
# model_onnx.eval()

# checkpoint_path = '/home/cristopher/Documents/SegTHRawS_training/model_training/models/Unet_mobilenet_v2_metrics_without_empty_masks/Unet_mobilenet_v2_epoch=149.ckpt'
# trained_model = ThermalModel.load_from_checkpoint(checkpoint_path=checkpoint_path,model=model,loss_fn=loss)

# onnx_model_path = checkpoint_path.replace('.ckpt','.onnx')
# onnx_model_path = checkpoint_path.replace('.ckpt','_sigmoid.onnx')
# model_onnx = trained_model.cpu()
# model_onnx.eval()

# torch_out = model_onnx(x)
# import warnings
# warnings.filterwarnings(category=FutureWarning,action='ignore')
# warnings.filterwarnings(category=torch.jit.TracerWarning,action='ignore')


# torch.onnx.export(model_onnx,                                   # model being run
#                   x,                                            # model input (or a tuple for multiple inputs)
#                   onnx_model_path,                              # where to save the model (can be a file or file-like object)
#                   export_params=True,                           # store the trained parameter weights inside the model file
#                   opset_version=15,                             # the ONNX version to export the model to
#                   do_constant_folding=True,                     # whether to execute constant folding for optimization
#                   input_names = ['input'],                      # the model's input names
#                   output_names = ['output'])




In [ ]:
# import matplotlib.image as mpimg

# image  = mpimg.imread('train_dataset/test/images/Australia_0_G0_(0, 384, 256, 640)_NIR_SWIR.png')

# image_model  = np.transpose(image,(2,0,1))[np.newaxis]


# trained_model(image_model)

In [ ]:
new_model

ThermalModel_sigmoid(
  (model): ThermalModel(
    (model): Unet(
      (encoder): MobileNetV2Encoder(
        (features): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): ReLU6(inplace=True)
          )
          (1): InvertedResidual(
            (conv): Sequential(
              (0): Conv2dNormActivation(
                (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
                (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
                (2): ReLU6(inplace=True)
              )
              (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
              (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
          )
        

In [ ]:
# torch.save(new_model,os.path.join(model_main_path,'trained_new_model'))

# # trained_model = ThermalModel_sigmoid.load_from_checkpoint(checkpoint_path=os.path.join(model_main_path,'trained_new_model'),model=model,loss_fn=loss)


# model_2 = torch.load(os.path.join(model_main_path,'trained_new_model'))


In [ ]:
# model_2.eval()

# trainer = pl.Trainer(
#     fast_dev_run=False,
#     accelerator="auto",
#     strategy="auto",
#     devices="auto",
#     num_nodes=1,
#     logger=logger,
#     callbacks=callbacks,
#     max_epochs=1,
#     min_epochs=1,
#     precision=precision #Mixed precision training
# )

# # Datamodule
# datamodule = ThermalDataModule(
#     images_path=DATA_PATH,
#     augmentation=augmentation,
#     preprocessing=preprocessing,
#     batch_size=16,
#     num_workers=os.cpu_count()
# )

# loss_2 = FocalLoss(mode= 'binary')
# loss_2.__name__ = 'focal_loss'

# # # LightningModule
# # lightning_model = ThermalModel(
# #     model=model,
# #     loss_fn=loss_2,
# #     optim_dict=optim_dict,
# #     lr=3e-4
# # )

# test_metrics = trainer.test(model=model_2,datamodule=datamodule)[0]
# # trainer.predict(model=model_2,datamodule=datamodule)

Using 16bit Automatic Mixed Precision (AMP)
Trainer already configured with model summary callbacks: [<class 'lightning.pytorch.callbacks.model_summary.ModelSummary'>]. Skipping setting a default `ModelSummary` callback.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 11/11 [00:00<00:00, 39.94it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_IoU            0.07021191716194153
       test_fbeta           0.12873856723308563
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


In [ ]:
# from training_utils import SegTHRawSModel,SegTHRawSDataModule
# import torch
# import lightning as pl
# import numpy as np
# import matplotlib.pyplot as plt

# import os

# import sys
# sys.path.insert(1,'..')
# from utils import read_binary_image

# model_path = '/home/cristopher/Documents/SegTHRawS/models/unet_smp_mobilenet_v2_fp16_focal_loss_smp_gamma2.0_lr_scheduler_geo_weakly/batch_size_16/seed_0/run_7/seed_0_epoch=78'

# model = torch.load(model_path)

# images_path = '/home/cristopher/Documents/SegTHRawS/datasets/train_geo_split_weakly_dataset/test/images'
# image_path = '/home/cristopher/Documents/SegTHRawS/datasets/train_geo_split_weakly_dataset/test/images/Chillan_Nevados_de_00_G0_(0, 384, 256, 640)_NIR_SWIR.bin'
# mask_path = '/home/cristopher/Documents/SegTHRawS/datasets/train_geo_split_weakly_dataset/test/masks/Chillan_Nevados_de_00_G0_(0, 384, 256, 640)_mask.bin'

# for idx,image_name in enumerate(os.listdir(images_path)):
#     image_path = os.path.join(images_path,image_name)
#     mask_path =  os.path.join(os.path.dirname(images_path),'masks',image_name.replace('NIR_SWIR','mask'))
#     image = torch.tensor(read_binary_image(image_path=image_path,dtype=np.float32,shape= [256,256,3]))
#     mask  = torch.tensor(read_binary_image(image_path=mask_path,dtype = np.float32, shape= [256,256,1]))

#     image_batch = np.transpose(image,(2,0,1))[np.newaxis]
#     mask_batch = np.transpose(mask,(2,0,1))[np.newaxis]


#     # predictions = model.test_step((image_batch,mask_batch),batch_idx=1)
#     predictions = model(image_batch.to('cuda'))

#     # test_masks_path = os.path.join(os.path.dirname(os.path.dirname(__file__)),'test_masks_comparison')
#     # os.makedirs(test_masks_path,exist_ok=True)

#     fig, ax = plt.subplots(1,3,figsize = (9,3))
#     prediction_mask = np.transpose(predictions[0].cpu().detach().numpy(),(1,2,0))

#     plt.suptitle(f' Test masks comparison', fontsize=14)
#     ax[0].imshow(image)
#     ax[1].imshow(mask)
#     ax[2].imshow(prediction_mask)
#     plt.tight_layout()
#     # plt.savefig(os.path.join(test_masks_path,f'test_mask_comparison_{self.test_idx+batch_idx}'))
#     plt.show()

#     if idx>3: break

# # model.eval()

# # trainer = pl.Trainer(
# #     fast_dev_run=False,
# #     accelerator="auto",
# #     strategy="auto",
# #     devices="auto",
# #     num_nodes=1,
# #     logger=logger,
# #     callbacks=callbacks,
# #     max_epochs=1,
# #     min_epochs=1,
# #     precision=precision #Mixed precision training
# # )

# # # Datamodule
# # datamodule = SegTHRawSDataModule(
# #     images_path=DATA_PATH,
# #     augmentation=augmentation,
# #     preprocessing=preprocessing,
# #     batch_size=1,
# #     num_workers=os.cpu_count()
# # )

# # loss_2 = FocalLoss(mode= 'binary')
# # loss_2.__name__ = 'focal_loss'

# # # # LightningModule
# # # lightning_model = ThermalModel(
# # #     model=model,
# # #     loss_fn=loss_2,
# # #     optim_dict=optim_dict,
# # #     lr=3e-4
# # # )



# # mask_prediction = trainer.test(model=model,datamodule=datamodule)[0]
# # # trainer.predict(model=model_2,datamodule=datamodule)

FileNotFoundError: [Errno 2] No such file or directory: '/home/cristopher/Documents/SegTHRawS/models/unet_smp_mobilenet_v2_fp16_focal_loss_smp_gamma2.0_lr_scheduler_geo_weakly/batch_size_16/seed_0/run_7/seed_0_epoch=78'

In [ ]:
# # model_ckpt = '/home/cristopher/Documents/SegTHRawS/models/seed_0_epoch=35.ckpt'
# model_ckpt = '/home/cristopher/Documents/SegTHRawS/models/seed_0_epoch=24.ckpt'

# from training_utils import SegTHRawSModel,SegTHRawSDataModule, SegTHRawSTrainModel, get_training_augmentation, get_preprocessing
# import lightning as pl
# from models import select_model
# from losses import select_loss
# import os
# import numpy as np
# import segmentation_models_pytorch as smp
# import matplotlib.pyplot as plt

# model = select_model(model_name = 'unet_smp',
#                 ENCODER = 'mobilenet_v2',
#                 ENCODER_WEIGHTS= 'imagenet',
#                 ACTIVATION=None)


# loss = select_loss(loss_name='focal_loss_smp',weakly=True)

# dataset_path = '/home/cristopher/Documents/SegTHRawS/datasets/train_geo_split_weakly_dataset'

# preprocessing_fn = smp.encoders.get_preprocessing_fn('mobilenet_v2', 'imagenet')



# model_2 = SegTHRawSTrainModel.load_from_checkpoint(checkpoint_path=model_ckpt,model=model,loss_fn = loss)

# model_2.eval()

# trainer = pl.Trainer(
#     fast_dev_run=False,
#     accelerator="auto",
#     strategy="auto",
#     devices="auto",
#     num_nodes=1,
#     # logger=logger,
#     # callbacks=callbacks,
#     max_epochs=1,
#     min_epochs=1,
#     precision='16-mixed', #Mixed precision training
    
# )

# # Datamodule
# datamodule = SegTHRawSDataModule(
#     images_path=dataset_path,
#     augmentation=get_training_augmentation(),
#     preprocessing=get_preprocessing(preprocessing_fn=preprocessing_fn),
#     batch_size=1,
#     num_workers=os.cpu_count()
# )


# # # LightningModule
# # lightning_model = ThermalModel(
# #     model=model,
# #     loss_fn=loss_2,
# #     optim_dict=optim_dict,
# #     lr=3e-4
# # )

# # test_metrics = trainer.test(model=model_2,datamodule=datamodule)[0]
# predictions = trainer.predict(model=model_2,datamodule=datamodule)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting DataLoader 0: 100%|██████████| 183/183 [00:01<00:00, 141.88it/s]


In [ ]:
# from train_constants import mean_imagenet_imgs, std_imagenet_imgs

# for idx,batch in enumerate(predictions):
#     image       = np.transpose(batch[0].cpu().detach().numpy()[0],(1,2,0))
#     mask        = np.transpose(batch[1].cpu().detach().numpy()[0],(1,2,0))
#     prediction_mask  = np.transpose(batch[2].cpu().detach().numpy()[0],(1,2,0))

#     image = image*np.array(std_imagenet_imgs) + np.array(mean_imagenet_imgs)
#     # image       = np.transpose(image,(1,2,0))

#     print(image.shape,mask.shape,prediction_mask.shape)


#     fig, ax = plt.subplots(1,3,figsize = (9,3))

#     plt.suptitle(f' Test masks comparison', fontsize=14)
#     ax[0].imshow(image)
#     ax[1].imshow(mask)
#     ax[2].imshow(prediction_mask)
#     plt.tight_layout()
#     # plt.savefig(os.path.join(test_masks_path,f'test_mask_comparison_{self.test_idx+batch_idx}'))
#     plt.show()

#     if idx>30: break
